In [21]:
import os
import sys
from typing import List

import pandas as pd
from qiskit_ibm_runtime import IBMRuntimeService, IBMSampler, SamplerResult
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
import numpy as np

In [29]:
def run_sampler(
    circuits: [QuantumCircuit], backend="ibmq_qasm_simulator", shots=1024, verbose=False
) -> SamplerResult:

    if backend != "statevector_simulator":
        for cirq in circuits:
            cirq.measure_all()

    service = IBMRuntimeService(auth="legacy", token="", instance="ibm-q/open/main")
    sampler_factory = IBMSampler(service=service, backend=backend)

    with sampler_factory(circuits=circuits) as sampler:
        result = sampler(circuit_indices=list(range(len(circuits))), shots=shots)
        if verbose:
            print(result)

        return result

In [30]:

def twolocal_tpl(
    nb_qubits: int,
    repetitions: int,
    rotation_blocks: [str],
    entanglement_blocks: str,
    entanglement,
    skip_final_rotation_layer: bool,
    name="Circuit_tpl",
    verbose=False,
) -> QuantumCircuit:
    circuit = TwoLocal(
        num_qubits=nb_qubits,
        rotation_blocks=rotation_blocks,
        entanglement_blocks=entanglement_blocks,
        entanglement=entanglement,
        reps=repetitions,
        insert_barriers=False,
        skip_final_rotation_layer=skip_final_rotation_layer,
        name=name,
    )

    if verbose:
        print(circuit.decompose())

    return circuit

In [38]:
def kernel_endpoint(
    circuit_tpl_id: [int],
    seed_x: [int],
    seed_y: [int],
    width: int = 4,
    layer: int = 1,
    backend: str = "ibmq_qasm_simulator",
    shots: int = 1024,
    verbose: bool = False,
) -> [str]:
    circuit_tpl = []
    for tpl_id in circuit_tpl_id:
        if tpl_id == 2:
            circuit_tpl.append(circuit_2(width=width, layer=layer, verbose=verbose))
        elif tpl_id == 5:
            circuit_tpl.append(circuit_5(width=width, layer=layer, verbose=verbose))
        elif tpl_id == 10:
            circuit_tpl.append(circuit_10(width=width, layer=layer, verbose=verbose))
        elif tpl_id == 18:
            circuit_tpl.append(circuit_18(width=width, layer=layer, verbose=verbose))
        else:
            print("Please chooce a circuit_tpl_id between [2, 5, 10, 18, X]")
            sys.exit(1)

    kernel_cirq = []
    for tpl in circuit_tpl:
        for index, _ in enumerate(seed_x):
            kernel_cirq.append(
                kernel_circuit(
                    circuit=tpl,
                    seed1=seed_x[index],
                    seed2=seed_y[index],
                    verbose=verbose,
                )
            )

    run = run_sampler(
        circuits=kernel_cirq, backend=backend, shots=shots, verbose=verbose
    )

    if verbose:
        print(
            "::set-output name={name}::{value}".format(name="KernelResult", value=run)
        )

    #file_fea = kernel_metadata(
    #    circuit_tpl_id=circuit_tpl_id,
    #    width=width,
    #    layer=layer,
    #    shots=shots,
    #    seed1=seed_x,
    #    seed2=seed_y,
    #    runtime_result=run,
    #)
    
    return run

In [39]:
    def kernel_flow(
        circuit_tpl_id: [int],
        width: int = 4,
        layer: int = 1,
        seed1: int = 42,
        seed2: int = 4242,
        matrix_size: List[int] = None,
        backend: str = "ibmq_qasm_simulator",
        shots: int = 1024,
        verbose: bool = False,
    ) -> [str]:
        seed_x = []
        seed_y = []

        if matrix_size is not None:
            if matrix_size[0] != matrix_size[1]:
                print(
                    """
                The coordinate have to be square.
                Ex. [2,2] or [5,5]
                """
                )
                sys.exit(1)
            for x_axe in range(matrix_size[0] + 1):
                for y_axe in range(matrix_size[1] + 1):
                    seed_x.append(x_axe)
                    seed_y.append(y_axe)
        else:
            seed_x.append(seed1)
            seed_y.append(seed2)

        return kernel_endpoint(
            circuit_tpl_id=circuit_tpl_id,
            width=width,
            layer=layer,
            seed_x=seed_x,
            seed_y=seed_y,
            backend=backend,
            shots=shots,
            verbose=verbose,
        )

In [57]:
def circuit_5(width=4, layer=1, verbose=False) -> QuantumCircuit:
    entangler_map = [(i, i - 1) for i in range(width - 1, 0, -1)]

    circuit = twolocal_tpl(
        nb_qubits=width,
        repetitions=layer,
        rotation_blocks=["rx", "rz"],
        entanglement_blocks="cx",
        entanglement=entangler_map,
        skip_final_rotation_layer=True,
        name="sim_circuit_2_%d_",
        verbose=verbose,
    )

    return circuit

In [58]:
def kernel_circuit(
    circuit: QuantumCircuit, seed1: int, seed2: int, verbose=False
) -> QuantumCircuit:
    template = circuit
    #verbose = True

    np.random.seed(seed1)
    x_axe = np.random.uniform(size=template.num_parameters)
    np.random.seed(seed2)
    y_axe = np.random.uniform(size=template.num_parameters)

    kernel_cirq = QuantumCircuit(template.num_qubits)
    kernel_cirq.append(
        template.bind_parameters(x_axe), list(range(template.num_qubits))
    )
    kernel_cirq.append(
        template.inverse().bind_parameters(y_axe), list(range(template.num_qubits))
    )

    if verbose:
        print(kernel_cirq.decompose())

    return kernel_cirq

In [59]:
run = kernel_flow(circuit_tpl_id=[5], matrix_size=[1,1])

In [62]:
for i in range(len(run.quasi_dists)):
    if run.quasi_dists[i].get("0000") is not None:
        print(run.quasi_dists[i]["0000"])
    elif run.quasi_dists[i].get("0") is not None:
        print(run.quasi_dists[i]["0"])
    else:
        print(0)

1.0
0.8408203125
0.8642578125
1.0
